In [1]:
import sys
sys.path.append("../../../../Eval_Framework/Pfiles/capacity_eval_lib")
sys.path.append("../../../../Eval_Framework/Pfiles/EXIT_chats")
sys.path.append("../../../../Eval_Framework/Pfiles/LDPC_lib")
from capacity_eval import *
from scipy.optimize import linprog


M = 10000
SNRs_a = np.array([[16.4,16.9,19.7],[14.5,14.5,17.9], [12.4,12.5,16.2],[10.4,10.4,14], [8.2,8.4,12.3], [6.2,6.2,10.2]])
pairs = np.array([(1,0),(2,0),(1,1),(1,2),(0,1),(0,2),(2,1)])

#SNRs_b = np.array([[15.5,16.2,13.1],[13.4,14.7,10.3],[11.5,12.3,8.2],[9.5,10.2,6.7],[7.1,8.1,4],[5.1,6.4,1.8], [2.6,4.1,-1.3]])
SNRs_b = np.array([[15., 20., 14.],[13., 18., 12.],[11., 16., 10.],[9., 14., 8.],[7., 12., 6],[4., 10., 3.],])


In [ ]:
# Proposed scheme
for g in SNRs_b:    
    print '\nSNRs:%s'%(str(g))
    for (Nb, Ns) in pairs:    
        gMAC = g[0]
        gBC = g[1]
        gHSI = g[2]
        cBC = eval_cap_BC(Nb, Ns, gBC, M)
        cHSI = eval_cap_HSI(Nb, Ns, gHSI, M)
#cMAC = eval_MAC_rand_phase(Nb, Ns, gMAC, M1 = 300, M2 = 300)

        cMAC = eval_MAC(Nb, Ns, gMAC, M)
#cMAC = np.array( [3.16984004,  2.5764985 ,  2.57675782,  1.51168887,  0.75597148,  0.75571739,  1.98341627])
#cBC = 3.4487448266329639
#cHSI = 1.613597068410328
        r3 = cMAC[0]
        r2As = cMAC[1]
        r2Bs = cMAC[2]
        r2b = cMAC[3]
        r1As = cMAC[4]
        r1Bs = cMAC[5]
        r1b = cMAC[6]

        Rbs = np.array([r2As,r2Bs]).min()
        Rbss = np.array([r3, cBC]).min()
        Rs = np.array([r1As, r1Bs]).min()
        Rb = np.array([r1b, cHSI]).min()




        c = [-1, -1]
        A = [[2, 1], [1, 1], [1,0], [0,1]]

        b = [Rbss, Rbs, Rs, Rb]
        x0_bounds = (0, None)
        x1_bounds = (0, None)
        res = linprog(c, A_ub=A, b_ub=b, bounds=(x0_bounds, x1_bounds))
        if Nb > 0:
            rb = res.values()[4][1]/Nb
        else:
            rb = 0
        if Ns > 0:    
            rs = res.values()[4][0]/Ns
        else:
            rs = 0
        T = 2*rb*Nb + rs*Ns
        print 'T=%.2f, Nb=%d, Ns=%d, rb=%0.2f, rs=%0.2f'%(T, Nb, Ns, rb, rs)
       

In [3]:
f = open("scheme_b_FTL",'w')
f.write("#2nd order cut-set MAC\ngMAC, \tgBC, \tgHSI, \tCap_fun\n")
fB = open("scheme_b_TL_B",'w')
fB.write("#B - TL\n SNR_MAC, \tSNR_BC, \tSNR_HSI, \tT\n")
fB1 = open("scheme_b_TL_B1",'w')
fB1.write("#B1 - TL\n SNR_MAC, \tSNR_BC, \tSNR_HSI, \tT\n")

  
for g in SNRs_b:    
    print '\nSNRs:%s'%(str(g))
    gMAC = g[0]
    gBC = g[1]
    gHSI = g[2]
    cap_MAC = cap_eval_Gauss(gMAC+3)*0.5
    cap_HSI = cap_eval_Gauss(gHSI+3)*0.5
    cap_BC = cap_eval_Gauss(gBC)
    cap_BC1 = cap_eval_Gauss(gBC) * 0.5
    cap_B = np.min([cap_MAC, cap_BC1])
    cap_B1 = np.min([cap_MAC, cap_HSI, cap_BC])
    
    print 'cap_B1:%.3f, \t cap_b:%.3f,  \t cap_fun:%.3f'%(cap_B1, cap_B, cap_MAC*2)
    f.write("%.1f, \t%.1f, \t%.1f, \t%.3f\n"%(gMAC, gBC, gHSI, cap_MAC*2))
    fB.write("%.1f, \t%.1f, \t%.1f, \t%.3f\n"%(gMAC, gBC, gHSI, cap_B))
    fB1.write("%.1f, \t%.1f, \t%.1f, \t%.3f\n"%(gMAC, gBC, gHSI, cap_B1))
f.close()
fB.close()
fB1.close()
    


SNRs:[ 15.  20.  14.]
cap_B1:2.838, 	 cap_b:3.001,  	 cap_fun:6.002

SNRs:[ 13.  18.  12.]
cap_B1:2.514, 	 cap_b:2.675,  	 cap_fun:5.351

SNRs:[ 11.  16.  10.]
cap_B1:2.195, 	 cap_b:2.354,  	 cap_fun:4.707

SNRs:[  9.  14.   8.]
cap_B1:1.882, 	 cap_b:2.037,  	 cap_fun:4.075

SNRs:[  7.  12.   6.]
cap_B1:1.580, 	 cap_b:1.730,  	 cap_fun:3.459

SNRs:[  4.  10.   3.]
cap_B1:1.158, 	 cap_b:1.294,  	 cap_fun:2.588


In [11]:
cap_eval_Gauss(4.5+3)

2.3858965294855752

In [19]:
m = 2
M = 8
N = 40
cap_MAC = np.zeros(40)
gMAC =  np.linspace(m,M,N)
f = open("FTL",'w')
f.write("#2nd order cut-set MAC\ngMAC, \tFTL, \trb\n")
for i in range(N):
    cap_MAC[i] = cap_eval_Gauss(gMAC[i]+1.76)
    f.write("%.3f, \t%.3f, \t%.3f\n"%(gMAC[i], cap_MAC[i], cap_MAC[i]/4))
f.close()

In [15]:
plt.plot(gMAC, cap_MAC)

In [16]:
plt.show()

In [7]:
cap_MAC

array([ 1.75567395,  1.79183386,  1.82836562,  1.86526361,  1.90252211,
        1.94013533,  1.97809738,  2.01640231,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])